In [92]:
import tensorflow as tf

import numpy as np
import os
import time

In [93]:
path_to_file = 'yaponskaya_classica.txt'

In [94]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 6315 characters


In [97]:
print(text[:200])


Басё

Луна – путеводный знак —

Просит: «Сюда пожалуйте!»

Дорожный приют в горах.


***

Наскучив долгим дождем,

Ночью сосны прогнали его…

Ветви в первом снегу.


***

Ирис на


In [98]:
text = text + text + text

In [99]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

76 unique characters


In [100]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [101]:
text_as_int, text, len(text_as_int), len(text)

(array([ 1,  0, 15, ...,  0,  1,  0]),
 '\r\nБасё\r\n\r\nЛуна – путеводный знак —\r\n\r\nПросит: «Сюда пожалуйте!»\r\n\r\nДорожный приют в горах.\r\n\r\n\r\n***\r\n\r\nНаскучив долгим дождем,\r\n\r\nНочью сосны прогнали его…\r\n\r\nВетви в первом снегу.\r\n\r\n\r\n***\r\n\r\nИрис на берегу.\r\n\r\nА вот другой – до чего похож! —\r\n\r\nОтраженье в воде.\r\n\r\n\r\nОтцу, потерявшему сына\r\n\r\nПоник головой, —\r\n\r\nСловно весь мир опрокинут, —\r\n\r\nПод снегом бамбук.\r\n\r\n\r\n***\r\n\r\nСыплются льдинки.\r\n\r\nСнега белая занавесь\r\n\r\nВ мелких узорах.\r\n\r\n\r\n***\r\n\r\nВечерним вьюнком\r\n\r\nЯ в плен захвачен… Недвижно\r\n\r\nСтою в забытьи.\r\n\r\n\r\nВ ответ на просьбу сочинить стихи\r\n\r\nВишни в весеннем расцвете.\r\n\r\nНо я – о горе!\xa0– бессилен открыть\r\n\r\nМешок, где спрятаны песни.\r\n\r\n\r\n***\r\n\r\nЛюди вокруг веселятся —\r\n\r\nИ только… Со склонов горы Хацусэ?\r\n\r\nГлядят невоспетые вишни.\r\n\r\n\r\n***\r\n\r\nБутоны вишневых цветов,\r\n\r\nСкорей

### train and target

In [102]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])




Б
а
с


In [103]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'\r\nБасё\r\n\r\nЛуна – путеводный знак —\r\n\r\nПросит: «Сюда пожалуйте!»\r\n\r\nДорожный приют в горах.\r\n\r\n\r\n***\r\n'
'\r\nНаскучив долгим дождем,\r\n\r\nНочью сосны прогнали его…\r\n\r\nВетви в первом снегу.\r\n\r\n\r\n***\r\n\r\nИрис на б'
'ерегу.\r\n\r\nА вот другой – до чего похож! —\r\n\r\nОтраженье в воде.\r\n\r\n\r\nОтцу, потерявшему сына\r\n\r\nПоник г'
'оловой, —\r\n\r\nСловно весь мир опрокинут, —\r\n\r\nПод снегом бамбук.\r\n\r\n\r\n***\r\n\r\nСыплются льдинки.\r\n\r\nСнег'
'а белая занавесь\r\n\r\nВ мелких узорах.\r\n\r\n\r\n***\r\n\r\nВечерним вьюнком\r\n\r\nЯ в плен захвачен… Недвижно\r\n\r\nС'


In [104]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [105]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\r\nБасё\r\n\r\nЛуна – путеводный знак —\r\n\r\nПросит: «Сюда пожалуйте!»\r\n\r\nДорожный приют в горах.\r\n\r\n\r\n***\r'
Target data: '\nБасё\r\n\r\nЛуна – путеводный знак —\r\n\r\nПросит: «Сюда пожалуйте!»\r\n\r\nДорожный приют в горах.\r\n\r\n\r\n***\r\n'


In [106]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [109]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [110]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [111]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [112]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [20]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [113]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [114]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 76) # (batch_size, sequence_length, vocab_size)


In [115]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (64, None, 128)           9728      
_________________________________________________________________
lstm_13 (LSTM)               (64, None, 1024)          4722688   
_________________________________________________________________
lstm_14 (LSTM)               (64, None, 1024)          8392704   
_________________________________________________________________
lstm_15 (LSTM)               (64, None, 1024)          8392704   
_________________________________________________________________
lstm_16 (LSTM)               (64, None, 1024)          8392704   
_________________________________________________________________
dense_13 (Dense)             (64, None, 76)            77900     
Total params: 29,988,428
Trainable params: 29,988,428
Non-trainable params: 0
_________________________________________

In [116]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 76), dtype=float32, numpy=
array([[ 7.29436522e-07, -5.42549287e-07, -7.06040919e-06, ...,
         3.65258279e-06, -5.08230278e-06, -1.34483107e-05],
       [ 3.57982572e-06, -2.49289769e-07, -4.41899792e-05, ...,
         1.29359860e-05, -1.54479494e-06, -2.47008356e-05],
       [ 1.19224842e-05,  1.99122223e-06, -1.06289735e-04, ...,
         1.18468015e-05,  6.65927473e-06, -2.45580886e-05],
       ...,
       [-8.35727085e-04,  2.72726174e-05, -8.24232935e-04, ...,
         4.30428190e-06,  1.11676077e-03, -1.90090458e-03],
       [-1.12729007e-03,  1.43716519e-04, -8.82806140e-04, ...,
         6.28548150e-06,  1.08218077e-03, -1.98064512e-03],
       [-1.42399408e-03,  2.89257674e-04, -9.68716864e-04, ...,
        -7.56794179e-06,  1.00139598e-03, -2.06588628e-03]], dtype=float32)>

In [117]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [118]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'аве\r\n\r\nПрозрачные мальки… Поймаешь —\r\n\r\nРастают без следа.\r\n\r\n\r\n***\r\n\r\nРосинки на горных розах.\r\n\r\nК'

Next Char Predictions: 
 'нэДщУз!Нб—ёИМоШ\r.ЖкАЛп?:А;–:зСМдэ\xa0Зд;УёжищъКБеЭтХч!бВмщ!и Н:Н«мМЗ–*ЭчддюиЧецЕЧ\rШаЦЛЕлУцеШзКю\rЮзНБыВЦ'


## Train the model

In [119]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 76)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.3305535


In [120]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [121]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=20,
    save_weights_only=True)

### Execute the training

In [122]:
EPOCHS = 200

In [123]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/200
2/2 [==============================] - 4s 347ms/step - loss: 4.3038
Epoch 2/200
2/2 [==============================] - 1s 356ms/step - loss: 4.1571
Epoch 3/200
2/2 [==============================] - 1s 352ms/step - loss: 3.9606
Epoch 4/200
2/2 [==============================] - 1s 360ms/step - loss: 4.6208
Epoch 5/200
2/2 [==============================] - 1s 356ms/step - loss: 3.8565
Epoch 6/200
2/2 [==============================] - 1s 357ms/step - loss: 3.4649
Epoch 7/200
2/2 [==============================] - 1s 365ms/step - loss: 3.4316
Epoch 8/200
2/2 [==============================] - 1s 361ms/step - loss: 3.4262
Epoch 9/200
2/2 [==============================] - 1s 361ms/step - loss: 3.4290
Epoch 10/200
2/2 [==============================] - 1s 358ms/step - loss: 3.4504
Epoch 11/200
2/2 [==============================] - 1s 353ms/step - loss: 3.4355
Epoch 12/200
2/2 [==============================] - 1s 354ms/step - loss: 3.4548
Epoch 13/200
2/2 [===================

## Generate text

In [124]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_200'

In [125]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [126]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (1, None, 128)            9728      
_________________________________________________________________
lstm_17 (LSTM)               (1, None, 1024)           4722688   
_________________________________________________________________
lstm_18 (LSTM)               (1, None, 1024)           8392704   
_________________________________________________________________
lstm_19 (LSTM)               (1, None, 1024)           8392704   
_________________________________________________________________
lstm_20 (LSTM)               (1, None, 1024)           8392704   
_________________________________________________________________
dense_14 (Dense)             (1, None, 76)             77900     
Total params: 29,988,428
Trainable params: 29,988,428
Non-trainable params: 0
_________________________________________

In [127]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [128]:
text_ = generate_text(model, start_string=u"Сегодня холодно с утра... ")
print(text_)

Сегодня холодно с утра... 

Откоть, спебет сверулет ся белок

Перило вололоя в подрек тебене…


***

Ива сволелний тены

Вом сводой ототной почал.


О витьй зок претнене

Простене верере.


***

И весно бестрей пореж.

Гудтя бестеть свее почассь,

По молкой худонь

Вспотет сслотея сочеть.


***


В ветеж я слетшей посне

Прешен со вореле в гороттут,

И тьчо веко мел кере!


***

Ма скоше вистретной петы

Просяд кекоя сореккий порал.


***

Чагок керкой зриство сточая вого 
